In [1]:
# import libraries
import geopandas as gpd
import pydeck as pdk
import os
from dotenv import load_dotenv
load_dotenv()

/opt/homebrew/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.12.1-CAPI-1.18.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


True

In [3]:
# load parcel data from project folder (Dropbox)
df_parcels = gpd.read_file(os.environ.get('LOCAL_PATH')+"Spatial Data/ny-parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")
# df_buildings = gpd.read_file(os.environ.get('LOCAL_PATH')+"Data/Building Footprints/NYC/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

In [4]:
# show length of dataframe
df_parcels.length

0        1921.544489
1        1200.788083
2        3741.111736
3         369.612212
4         502.254754
            ...     
42292     207.048801
42293     159.618447
42294     656.379825
42295     213.730693
42296     279.662391
Length: 42297, dtype: float64

In [ ]:
# subset dataframe

In [6]:
# set CRS to WGS 84
df_parcels = df_parcels.to_crs(epsg=4326)
# df_buildings = df_buildings.to_crs(epsg=4326)

In [7]:
# extract lat and lon from the centerpoint
df_parcels['center_point'] = df_parcels['geometry'].centroid
df_parcels['lon'] = df_parcels['center_point'].x
df_parcels['lat'] = df_parcels['center_point'].y

/var/folders/9g/r0ctqhfj26l910sgbwcdndq00000gn/T/ipykernel_5115/1599866429.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_parcels['center_point'] = df_parcels['geometry'].centroid


In [8]:
# view df (optional)
print(df_parcels['geometry'].head())
print(df_parcels['center_point'].head())
print(df_parcels['center_point'].head())
print(df_parcels['lon'].head())
print(df_parcels['lat'].head())

0    POLYGON ((-74.03823 40.69837, -74.03823 40.698...
1    MULTIPOLYGON (((-74.04270 40.69006, -74.04263 ...
2    POLYGON ((-74.02240 40.68442, -74.02404 40.683...
3    POLYGON ((-74.00855 40.70236, -74.00864 40.702...
4    POLYGON ((-74.00940 40.70242, -74.01023 40.703...
Name: geometry, dtype: geometry
0    POINT (-74.04087 40.69871)
1    POINT (-74.04536 40.68992)
2    POINT (-74.01909 40.68877)
3    POINT (-74.00791 40.70291)
4    POINT (-74.00915 40.70318)
Name: center_point, dtype: geometry
0    POINT (-74.04087 40.69871)
1    POINT (-74.04536 40.68992)
2    POINT (-74.01909 40.68877)
3    POINT (-74.00791 40.70291)
4    POINT (-74.00915 40.70318)
Name: center_point, dtype: geometry
0   -74.040870
1   -74.045362
2   -74.019093
3   -74.007907
4   -74.009151
Name: lon, dtype: float64
0    40.698711
1    40.689919
2    40.688773
3    40.702907
4    40.703185
Name: lat, dtype: float64


In [ ]:
# display parcels as points
points = pdk.Layer(
    'ScatterplotLayer',
    data=df_parcels,
    get_position='[lon, lat]',
    get_radius=10,
    get_fill_color=[255, 0, 0],
    auto_highlight=True,
    pickable=True
)

# heatmap
heat = pdk.Layer(
    "HeatmapLayer",
    data=df_parcels,
    get_position=["lon", "lat"],
    #aggregation=pdk.types.String("MEAN"),
    aggregation=pdk.types.String("SUM"),
    get_weight=1, # Assuming equal weight for all points
    #threshold=1,
    opacity=0.9,
)

# hexagon
hex = pdk.Layer(
    "HexagonLayer",
    data=df_parcels,
    get_position=["lon", "lat"],
    radius=50,
    opacity=0.9,
    pickable=True
)

# Create the map
view_state = pdk.ViewState(
    latitude=40.76,
    longitude=-74,
    zoom=11,
    max_zoom=20,
    min_zoom=10
)

r = pdk.Deck(
    layers=[hex],
    initial_view_state=view_state
)

# Display the visualization in the notebook (warning: increases notebook size significantly)
# r.show()

# save the visualization to an HTML file (git-ignored)
r.to_html('parcels-map.html')